In [1]:
import pandas as pd

iter = 4
df = pd.read_csv(f'crowdsourced-robotinder-demo/flag_data{iter}.csv')
if iter >2:
    df.columns = ['env_name', 'user_choice', 'left_video', 'right_video', 'time', 'user_id']
else:
    df.columns = ['env_name', 'user_choice', 'left_video', 'right_video', 'time']

print(df.head(5))
print(f'total number of samples: {len(df.index)}')


                  env_name user_choice  \
0     ShadowHandBlockStack    Not Sure   
1   ShadowHandLiftUnderarm        Left   
2       ShadowHandScissors        Left   
3      ShadowHandBottleCap       Right   
4  ShadowHandCatchUnderarm    Not Sure   

                                          left_video  \
0  processed-data/itr4/ShadowHandBlockStack/Shado...   
1  processed-data/itr4/ShadowHandLiftUnderarm/Sha...   
2  processed-data/itr4/ShadowHandScissors/ShadowH...   
3  processed-data/itr4/ShadowHandBottleCap/Shadow...   
4  processed-data/itr4/ShadowHandCatchUnderarm/Sh...   

                                         right_video                 time  \
0  processed-data/itr4/ShadowHandBlockStack/Shado...  2023-02-05-17-06-50   
1  processed-data/itr4/ShadowHandLiftUnderarm/Sha...  2023-02-06-15-42-40   
2  processed-data/itr4/ShadowHandScissors/ShadowH...  2023-02-06-15-43-13   
3  processed-data/itr4/ShadowHandBottleCap/Shadow...  2023-02-06-15-43-27   
4  processed-data/itr4/Sh

In [2]:
# select 17 envs
env_list = ['ShadowHand', 'ShadowHandCatchAbreast', 'ShadowHandOver', 'ShadowHandBlockStack', 'ShadowHandCatchUnderarm',
        'ShadowHandCatchOver2Underarm', 'ShadowHandBottleCap', 'ShadowHandLiftUnderarm', 'ShadowHandTwoCatchUnderarm', 'ShadowHandDoorOpenInward',
        'ShadowHandDoorOpenOutward', 'ShadowHandDoorCloseInward', 'ShadowHandGraspAndPlace', 'ShadowHandPushBlock',
        'ShadowHandScissors', 'ShadowHandPen', 'ShadowHandSwitch']

## Across-user Analysis

In [106]:
import numpy as np
import copy 

pref_list = {env: [0 for _ in np.arange(0, 10)] for env in env_list}
choice_nums = {env: [0 for _ in np.arange(0, 10)] for env in env_list}  # as the denominator
total_cnt = 0
print(pref_list, len(pref_list))
for index, row in df.iterrows():
    if row['user_choice'] in ['Left', 'Right']:
        total_cnt += 1
        if iter >2:
            env_name = row['left_video'].split('/')[2]  # processed-data/itr3/ShadowHandBottleCap/ShadowHandBottleCap_ppo_38_20230120185342_4000_video-episode-1.mp4
            left_seed = int(row['left_video'].split('/')[3].split('_')[2])
            right_seed = int(row['right_video'].split('/')[3].split('_')[2])
        else:
            env_name = row['left_video'].split('/')[1]  # processed-data/ShadowHandBottleCap/ShadowHandBottleCap_ppo_38_20230120185342_4000_video-episode-1.mp4
            left_seed = int(row['left_video'].split('/')[2].split('_')[2])
            right_seed = int(row['right_video'].split('/')[2].split('_')[2])            
        choice_nums[env_name][left_seed-iter*10] += 1
        choice_nums[env_name][right_seed-iter*10] += 1
        if row['user_choice'] == 'Left':
            pref_list[env_name][left_seed-iter*10] += 1
            pref_list[env_name][right_seed-iter*10] -= 1  # unprefer
        elif row['user_choice'] == 'Right':
            pref_list[env_name][right_seed-iter*10] += 1
            pref_list[env_name][left_seed-iter*10] -= 1  # unprefer

norm_pref_list = copy.deepcopy(pref_list)  # normalize by the total number of choices for env-seed pair
for env in env_list:
    for i in np.arange(0, 10):
        if choice_nums[env][i] == 0:
            norm_pref_list[env][i] = 0
        else:
            norm_pref_list[env][i] = pref_list[env][i] / (choice_nums[env][i])  # +1 avoid zero division

            
for env in env_list:
    for i in np.arange(0, 10):
        pref_list[env][i] = pref_list[env][i] / total_cnt
print(pref_list)

print(norm_pref_list)


{'ShadowHand': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandCatchAbreast': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandOver': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandBlockStack': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandCatchUnderarm': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandCatchOver2Underarm': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandBottleCap': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandLiftUnderarm': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandTwoCatchUnderarm': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandDoorOpenInward': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandDoorOpenOutward': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandDoorCloseInward': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandGraspAndPlace': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandPushBlock': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandScissors': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandPen': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ShadowHandSwitch': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]} 17
{'ShadowHand': [-0.004228329809725159

In [82]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

sns.set(font_scale=1.3)

for env in env_list:
    sns.barplot(x=np.arange(10), y=pref_list[env])
    # sns.displot(d ata=pref_list[env], bins=10, color="dodgerblue",  kde=True)
    plt.xlabel('Model Index')
    plt.ylabel('Human Preference Score')
    plt.title(f'{env}')
    # autoscale the plot to include all bars and labels
    # plt.autoscale(tight=False)
    os.makedirs(f'./plot/preference/iter{iter}', exist_ok=True)
    plt.savefig(f'./plot/preference/iter{iter}/{env}.png', dpi=300, bbox_inches = 'tight')
    # plt.show()
    plt.clf()


<Figure size 432x288 with 0 Axes>

In [83]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

sns.set(font_scale=1.3)

for env in env_list:
    sns.barplot(x=np.arange(10), y=norm_pref_list[env])
    # sns.displot(d ata=pref_list[env], bins=10, color="dodgerblue",  kde=True)
    plt.xlabel('Model Index')
    plt.ylabel('Human Preference Score')
    plt.title(f'{env}')
    # autoscale the plot to include all bars and labels
    # plt.autoscale(tight=False)
    os.makedirs(f'./plot/preference/iter{iter}', exist_ok=True)
    plt.savefig(f'./plot/preference/iter{iter}/{env}_norm.png', dpi=300, bbox_inches = 'tight')
    # plt.show()
    plt.clf()


<Figure size 432x288 with 0 Axes>

## Per-user Analysis

In [3]:
dfs = [y for x, y in df.groupby('user_id')]
users = [x for x, y in df.groupby('user_id')]

print(f'total number of users: {len(dfs)}') 
for u, d in zip(users, dfs):
    # print(d.head(5))
    print(f'user: {u}, number of samples: {len(d.index)}')

per_user_df = dfs[0]


total number of users: 6
user: Alen, number of samples: 30
user: Allen, number of samples: 94
user: Xiao Ma, number of samples: 157
user: Yuqi Nie, number of samples: 49
user: yuanpei, number of samples: 181
user: zhd, number of samples: 194


In [111]:
import numpy as np
import copy 


for idx, (per_user_df, user) in enumerate(zip(dfs, users)):
    pref_list = {env: [0 for _ in np.arange(0, 10)] for env in env_list}
    choice_nums = {env: [0 for _ in np.arange(0, 10)] for env in env_list}  # as the denominator
    total_cnt = 0
    print(len(pref_list))
    for index, row in per_user_df.iterrows():
        if row['user_choice'] in ['Left', 'Right']:
            total_cnt += 1
            if iter >2:
                env_name = row['left_video'].split('/')[2]  # processed-data/itr3/ShadowHandBottleCap/ShadowHandBottleCap_ppo_38_20230120185342_4000_video-episode-1.mp4
                left_seed = int(row['left_video'].split('/')[3].split('_')[2])
                right_seed = int(row['right_video'].split('/')[3].split('_')[2])
            else:
                env_name = row['left_video'].split('/')[1]  # processed-data/ShadowHandBottleCap/ShadowHandBottleCap_ppo_38_20230120185342_4000_video-episode-1.mp4
                left_seed = int(row['left_video'].split('/')[2].split('_')[2])
                right_seed = int(row['right_video'].split('/')[2].split('_')[2])            
            choice_nums[env_name][left_seed-iter*10] += 1
            choice_nums[env_name][right_seed-iter*10] += 1
            if row['user_choice'] == 'Left':
                pref_list[env_name][left_seed-iter*10] += 1
                pref_list[env_name][right_seed-iter*10] -= 1  # unprefer
            elif row['user_choice'] == 'Right':
                pref_list[env_name][right_seed-iter*10] += 1
                pref_list[env_name][left_seed-iter*10] -= 1  # unprefer

    # print(user, pref_list, total_cnt)
    for env in env_list:
        for i in np.arange(0, 10):
            pref_list[env][i] = pref_list[env][i] / total_cnt  # the individual total count 
    # print(user, pref_list, total_cnt)


    sns.set(font_scale=1.3)

    for env in env_list:
        plt.figure(figsize=(5, 4))
        plt.ylim(-0.05, 0.05)
        sns.barplot(x=np.arange(10), y=pref_list[env])
        # sns.displot(d ata=pref_list[env], bins=10, color="dodgerblue",  kde=True)
        plt.xlabel('Model Index')
        plt.ylabel('Human Preference Score')
        plt.title(f'{env}: Labler {idx+1}')
        # autoscale the plot to include all bars and labels
        # plt.autoscale(tight=False)
        os.makedirs(f'./plot/preference/iter{iter}/per_user/{user}', exist_ok=True)
        plt.savefig(f'./plot/preference/iter{iter}/per_user/{user}/{env}.png', dpi=300, bbox_inches = 'tight')
        # plt.show()
        plt.clf()


17
17


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


17
17
17


<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

...

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

In [91]:
all_envs = df.env_name.unique()
for env in all_envs:
    # print(env, df.loc[df['env_name']==env])
    pass
all_envs.sort()
print(all_envs)


['ShadowHand' 'ShadowHandBlockStack' 'ShadowHandBottleCap'
 'ShadowHandCatchAbreast' 'ShadowHandCatchOver2Underarm'
 'ShadowHandCatchUnderarm' 'ShadowHandDoorCloseInward'
 'ShadowHandDoorOpenInward' 'ShadowHandDoorOpenOutward'
 'ShadowHandGraspAndPlace' 'ShadowHandLiftUnderarm' 'ShadowHandOver'
 'ShadowHandPen' 'ShadowHandPushBlock' 'ShadowHandScissors'
 'ShadowHandSwitch' 'ShadowHandTwoCatchUnderarm']
